In [ ]:
# Считайте файл согласно вашему варианту и подсчитайте частоту каждого слова в тексте. 
# В результирующем файле выведите полученные данные в порядке убывания:

# word1:freq1
# word2:freq2
# word3:freq3
# -----------
# wordN:freqN

In [5]:
filename = "text_1_var_17"

with open(filename) as f:
    # cчитывает из файла все строки в список и возвращает его.
    lines = f.readlines()
    
# создаем словарь
d = dict()

# итерация по списку
for i in lines:
    # итерация по строкам 
    for j in (i.strip()
         .replace("!", " ")
         .replace("?", " ")
         .replace(",", " ")
         .replace(".", " ")
         .strip()).split():
        try:
            d[j] += 1
        except:
            d.update({j : 1})

d = dict(sorted(d.items(), key=lambda x: x[1], reverse = True))
print(d)

with open(filename+"result", "w") as f:
    for k in d:
        f.write(f"{k}:{d[k]}\n")

{'kytkWJBIBVAo': 134, 'LPQjCwSVe': 130, 'sSasUMyQpX': 129, 'UsmafIVvNUrq': 125, 'aooPAAmwvPy': 124, 'VnZs': 123, 'QzSyFatiW': 121, 'lmszdIAQtH': 120, 'xnUjJxAjU': 120, 'kKVgPPPU': 120, 'YUiBUAgcDC': 119, 'jCyD': 118, 'baK': 114, 'wYfSm': 114, 'gTkUFAu': 114, 'ZTBO': 113, 'CihB': 112, 'vFlcyb': 111, 'nwPYZZtAYjIb': 98, 'wWY': 98}
